Supplement A_Guide_to_Running_Tensorflow_Models_on_Android/Instructions.ipynb with more details    
Add comments on code

# Train the model

There is another example:    
https://github.com/guihongwan/PlaygroundOfTensorFlow/#Tutorials#Building a Convolutional Neural Network for MNIST.ipynb

In [13]:
# Python 3.6.0
# tensorflow 1.1.0
# Keras 2.0.4

import os
import os.path as path

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

MODEL_NAME = 'mnist_convnet'
EPOCHS = 1
BATCH_SIZE = 128

In [14]:
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    #print(x_train.shape)#(60000, 28, 28)
    #print(x_test.shape)#(10000, 28, 28)
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) # 784
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255 #the minumum value of color is 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, 10)
    #print(y_test) #[7 2 1 ... 4 5 6]
    y_test = keras.utils.to_categorical(y_test, 10)
    #print(y_test)
    #[[0. 0. 0. ... 1. 0. 0.]
    #[0. 0. 1. ... 0. 0. 0.]
    #[0. 1. 0. ... 0. 0. 0.]
    
    return x_train, y_train, x_test, y_test

def build_model():
    model = Sequential() #a linear stack of layers https://keras.io
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 64] 
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, \
            padding='same', activation='relu', \
            input_shape=[28, 28, 1]))
    # 28*28*64
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 14*14*64

    model.add(Conv2D(filters=128, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 14*14*128
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 7*7*128

    model.add(Conv2D(filters=256, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 7*7*256
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 4*4*256, because padding='same'

    model.add(Flatten()) #[bathsize,4*4*256]
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

def train(model, x_train, y_train, x_test, y_test):
    model.compile(loss=keras.losses.categorical_crossentropy, \
                  optimizer=keras.optimizers.Adadelta(), \
                  metrics=['accuracy'])

    model.fit(x_train, y_train, \
              batch_size=BATCH_SIZE, \
              epochs=EPOCHS, \
              verbose=1, \
              validation_data=(x_test, y_test))
    
# export model for andorid
def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt') #writes a graph proto to a file

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')
    
    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")
    
    #Optimize for inference
    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

In [15]:
def main():
    if not path.exists('out'):
        os.mkdir('out')

    x_train, y_train, x_test, y_test = load_data()

    model = build_model()

    train(model, x_train, y_train, x_test, y_test)

    export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")

In [16]:
if __name__ == '__main__':
    main()

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 531s 9ms/step - loss: 0.5903 - acc: 0.8165 - val_loss: 0.1883 - val_acc: 0.9372
INFO:tensorflow:Restoring parameters from out/mnist_convnet.chkp
INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.
graph saved!



# App Architecture

List<Classifier> mClassifiers:    
loadModel(): mClassifiers.add(TensorFlowClassifier.create())   
    
final Classification res = classifier.recognize(pixels);    

res is the result.

# Classifier
public interface <span class="mark">Classifier</span> {    
    String name();    
    Classification recognize(final float[] pixels);    
}    
public class <span class="mark">TensorFlowClassifier</span> implements Classifier


# TensorFlowClassifier

<span class="mark">TensorFlowInferenceInterface</span>:used as the window between android and tensorflow native C++    

create():    
c.tfHelper = new TensorFlowInferenceInterface(assetManager, <span class="mark">modelPath</span>);    

recognize(final float[] pixels):    
tfHelper.<span class="mark">feed</span>(inputName, pixels, 1, inputSize, inputSize, 1);      
tfHelper.<span class="mark">run</span>(outputNames);    
tfHelper.<span class="mark">fetch</span>(outputName, output);   

# Code of TensorFlowInferenceInterface
https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/android
